# Neighborlist generation

Generating the neighbor list is not a trivial exercise. It can be slow and difficult to classify exactly what is and isn't a neighbor.

In [ ]:
import ase.db
from ase.neighborlist import NeighborList
from ase.data import covalent_radii
from ase.io import write

In [ ]:
# Connect the ase-db.
db = ase.db.connect('../../data/solar.db')
atoms = list(db.select())
# random.shuffle(atoms)

# Compile a list of atoms and target values.
alist = []
targets = []
for row in atoms:
    try:
        targets.append(row.Energy)
        alist.append(row.toatoms())
    except AttributeError:
        continue

# Analyze the size of molecules in the db.
print('pulled {} molecules from db'.format(len(alist)))
size = []
for a in alist:
    size.append(len(a))

print('min: {0}, mean: {1:.0f}, max: {2} molecule size'.format(
    min(size), sum(size)/len(size), max(size)))

In [ ]:
atoms = alist[0]

cutoffs = [covalent_radii[a.number] for a in atoms]
nl = NeighborList(
    cutoffs, skin=0.3, sorted=False, self_interaction=False, bothways=True)

nl.build(atoms)

neighborlist = {}
for i, _ in enumerate(atoms):
    neighborlist[i] = nl.get_neighbors(i)[0]

print(neighborlist)